In [1]:
!pip install -q torch \
    transformers \
    accelerate \
    bitsandbytes \
    langchain \
    sentence-transformers \
    faiss-gpu \
    openpyxl \
    pacmap \
    python-dotenv \
    pandas \
    datasets \
    matplotlib \
    ragatouille \
    langchain-openai \
    ragas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 17.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.9/163.9 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from tqdm.notebook import tqdm
import pandas as pd
from typing import Optional, List, Tuple
from datasets import Dataset
import matplotlib.pyplot as plt
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import datasets
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy
from datetime import datetime
import ragas
from ragatouille import RAGPretrainedModel

pd.set_option(
    "display.max_colwidth", None
)  # this will be helpful when visualizing retriever outputs

In [3]:
import torch
torch.cuda.device_count()

1

In [4]:
QUESTION = 'I have a scenario for a disorder can say what is the sub type based on senario? the senario is: A 30-year-old man began using prescription painkillers for a back injury and now uses opioids daily, unable to stop despite severe health and legal issues.'

# Embedding model for RAG first retrieval
ENCODER = "thenlper/gte-small"

# Config for the optional reranking step
NUM_RETRIEVED_DOCS = 4
RERANKER = 'colbert-ir/colbertv2.0'
NUM_RERANKED_DOCS = 3

# The LLM we will use as "reader"
# LLM = 'gpt-3.5-turbo-0125'   "microsoft/Phi-3-mini-128k-instruct"

In [5]:
import re

def clean_text(text):
    # Remove headers, footers, and unnecessary sections
    text = remove_headers_and_footers(text)
    text = remove_copyright_info(text)
    text = remove_redundant_section_headers(text)

    # Normalize spaces and remove non-ASCII characters
    text = re.sub(r'\n+', '\n', text)  # Normalize multiple newlines
    text = re.sub(r'\s+', ' ', text)  # Remove excessive whitespace
    text = re.sub(r'[^\x00-\x7F]+', '', text)  # Remove non-ASCII characters

    # Remove specific patterns
    text = re.sub(r'(DSM-5TM)', '', text, flags=re.IGNORECASE)
    text = re.sub(r'(American Psychiatric Association)', '', text, flags=re.IGNORECASE)

    return text.strip()

def remove_headers_and_footers(text):
    lines = text.splitlines()
    clean_lines = []
    for line in lines:
        if not line.strip().startswith('DIAGNOSTIC AND STATISTICAL MANUAL') and \
           not line.strip().startswith('DSM-5') and \
           not line.strip().startswith('American Psychiatric Association'):
            clean_lines.append(line)
    return '\n'.join(clean_lines)

def remove_copyright_info(text):
    copyright_patterns = [
        r'Copyright © [0-9]{4} American Psychiatric Association',
        r'ALL RIGHTS RESERVED',
        r'Correspondence regarding copyright permissions should be directed to DSM Permissions',
        r'Manufactured in the United States of America on acid-free paper',
        r'Library of Congress Cataloging-in-Publication Data'
    ]
    for pattern in copyright_patterns:
        text = re.sub(pattern, '', text, flags=re.IGNORECASE)
    return text

def remove_redundant_section_headers(text):
    section_patterns = [
        r'Section I[\s\S]*?Section II',
        r'Section III[\s\S]*?Appendix'
    ]
    for pattern in section_patterns:
        text = re.sub(pattern, '', text, flags=re.IGNORECASE)
    return text

# Load and clean the text
# with open('DSM5.txt', 'r') as file:
#     raw_text = file.read()

# cleaned_text = clean_text(raw_text)

# # Save the cleaned text to a new file
# with open('DSM5_cleaned.txt', 'w') as file:
#     file.write(cleaned_text)

# print("Text cleaning and preprocessing completed.")

In [7]:
with open('/content/DSM5 ENGLISH.txt','r') as file:
  raw_text = file.read()
  text = clean_text(raw_text)
with open('/content/final_disease_subdisease_summaries.txt','r') as file:
  text2 = file.read()
ds = [text, text2]

In [8]:
ds2 = [text2]

In [9]:
from langchain.docstore.document import Document as LangchainDocument

LANGCHAIN_RAW_KNOWLEDGE_BASE = [
    LangchainDocument(page_content=doc)
    for doc in ds
]

In [10]:
LANGCHAIN_DISEASES_KNOWLEDGE_BASE = [
    LangchainDocument(page_content=doc)
    for doc in ds2
]

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer
import tqdm

CHUNK_SIZE = 256 # because we know it's the max seq length from the embedding model
MARKDOWN_SEPARATORS = [
    "\n#{1,6} ",
    "\n\n",
    "\n",
    " ",
    "",
]

text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    AutoTokenizer.from_pretrained(ENCODER),
    chunk_size=CHUNK_SIZE,
    chunk_overlap=int(CHUNK_SIZE / 10),
    add_start_index=True,
    strip_whitespace=True,
    separators=MARKDOWN_SEPARATORS,
)

# same as before, just with the new tokenizer
docs_processed = []
for doc in tqdm.tqdm(LANGCHAIN_RAW_KNOWLEDGE_BASE):
    docs_processed += text_splitter.split_documents([doc])


diseases_docs_processed = []
for doc in tqdm.tqdm(LANGCHAIN_DISEASES_KNOWLEDGE_BASE):
    diseases_docs_processed += text_splitter.split_documents([doc])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

100%|██████████| 1/1 [00:00<00:00, 26.09it/s]


In [12]:
# Remove duplicates
unique_texts = set()
docs_processed_unique = []
for doc in docs_processed:
    if doc.page_content not in unique_texts:
        unique_texts.add(doc.page_content)
        docs_processed_unique.append(doc)

In [13]:
print(len(docs_processed))
print(len(docs_processed_unique))

5023
5021


In [14]:
# Remove duplicates
unique_texts_diseases = set()
diseases_docs_processed_unique = []
for doc in diseases_docs_processed:
    if doc.page_content not in unique_texts_diseases:
        unique_texts_diseases.add(doc.page_content)
        diseases_docs_processed_unique.append(doc)

In [15]:
print(len(diseases_docs_processed))
print(len(diseases_docs_processed_unique))

16
16


In [16]:
def get_cur_time():
  now = datetime.now()
  return now.isoformat()

embedding_model = HuggingFaceEmbeddings(
    model_name=ENCODER,
    multi_process=True,
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},  # set True for cosine similarity
)

start = get_cur_time()
print(f'Starting embedding at {start}')
KNOWLEDGE_VECTOR_DATABASE = FAISS.from_documents(
    docs_processed_unique, embedding_model, distance_strategy=DistanceStrategy.COSINE
)

end = get_cur_time()
print(f'Finished embedding at {end}')

modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/68.1k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/66.7M [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Starting embedding at 2024-08-16T18:02:00.971037
Finished embedding at 2024-08-16T18:02:15.359291


In [17]:
start = get_cur_time()
print(f'Starting embedding at {start}')
KNOWLEDGE_VECTOR_DATABASE_1 = FAISS.from_documents(
    diseases_docs_processed_unique, embedding_model, distance_strategy=DistanceStrategy.COSINE
)

end = get_cur_time()
print(f'Finished embedding at {end}')

Starting embedding at 2024-08-16T18:02:15.375915
Finished embedding at 2024-08-16T18:02:19.796133


In [18]:
# embed a user query in the same space
query_vector = embedding_model.embed_query(QUESTION)

In [19]:
KNOWLEDGE_VECTOR_DATABASE_1.save_local('test1')

In [20]:
KNOWLEDGE_VECTOR_DATABASE.save_local('test')

In [21]:
!pip install langchain-huggingface transformers -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 82.9 MB/s eta 0:00:00


In [23]:
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_id = "microsoft/Phi-3-mini-128k-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map='cuda:0', trust_remote_code = True)
pipe = pipeline(
    "text-generation", model=model, tokenizer=tokenizer, max_new_tokens=128
)
hf_pipe = HuggingFacePipeline(pipeline=pipe)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [36]:
class Reader:

    def __init__(self):#, openai_api_key: str):

        self.llm = hf_pipe#ChatOpenAI(openai_api_key=openai_api_key, model=LLM)
        #self.openai_api_key = openai_api_key
        self.output_parser = StrOutputParser()

    def base_llm_answer(self, question: str) -> str:
        base_prompt = ChatPromptTemplate.from_messages([
            ("system", "You are a helpful assistant."),
            ("user", "{question}")
        ])
        chain = base_prompt | self.llm | self.output_parser
        answer = chain.invoke({'question': question})
        return answer

    def rag_llm_answer(self, question: str, context: Optional[str] = None) -> str:
        rag_prompt = ChatPromptTemplate.from_messages([
            ("system", """Using the information contained in the context,
            give a comprehensive answer to the question.
            Respond only to the question asked, response should be concise and relevant to the question.
            Provide the number of the source document when relevant.
            If the answer cannot be deduced from the context, do not give an answer."""),
            ("user", """Context:
            {context}
            ---
            Now here is the question you need to answer.

            Question: {question}""")
        ])
        chain = rag_prompt | self.llm | self.output_parser
        answer = chain.invoke({'context': context, 'question': question})
        return answer

# Let's initialize our Reader class, and use it to answer our question based on previously retrieved documents! First for a small test.

reader = Reader()#openai_api_key=openai_api_key)


In [37]:
# let's write some convenience functions

def retrieved_docs_to_prompt_context(retrieved_docs):
    retrieved_docs_text = [
        doc.page_content for doc in retrieved_docs
    ]  # we only need the text of the documents
    context = "\nExtracted documents:\n"
    context += "".join(
        [f"Document {str(i)}:::\n" + doc for i, doc in enumerate(retrieved_docs_text)]
    )
    return retrieved_docs_text

def rag_pipeline(question, reranker=None, return_context=False):
    # Step 1: Enhance the question using the base_llm_answer method.
    enhanced_question = question

    # Step 2: Perform the retrieval and optionally reranking
    retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(
        query=enhanced_question, k=NUM_RETRIEVED_DOCS)

    # No need to print the number of retrieved documents
    prompt_context = retrieved_docs_to_prompt_context(retrieved_docs)

    if reranker:
        relevant_docs_reranked = reranker.rerank(enhanced_question,
                                                 [doc.page_content for doc in retrieved_docs],
                                                 k=NUM_RERANKED_DOCS)

        # No need to print the reranking process
        relevant_docs_reranked_text = [doc["content"] for doc in relevant_docs_reranked]

        # Step 3: Use the refined question to get a final RAG answer
        rag_answer = reader.rag_llm_answer(enhanced_question, relevant_docs_reranked_text)
    else:
        # Step 3: Use the refined question to get a final RAG answer
        rag_answer = reader.rag_llm_answer(enhanced_question, prompt_context)

    # Only return the answer (and context if requested)
    if return_context:
        return (rag_answer, retrieved_docs)
    else:
        return rag_answer


def classificationn_pipeline(question, reranker=None, return_context = False):
    retrieved_docs = KNOWLEDGE_VECTOR_DATABASE_1.similarity_search(
    query=question, k=1)
    print(f'=> Retrieved {len(retrieved_docs)} docs')
    prompt_context = retrieved_docs_to_prompt_context(retrieved_docs)
    if reranker:
       print(f"=> Reranking documents...")
       relevant_docs_reranked = reranker.rerank(question,
                                               [doc.page_content for doc in retrieved_docs],
                                               k=1)
       print(f'=> After reranking, {len(relevant_docs_reranked)} docs')
       relevant_docs_reranked_text = [doc["content"] for doc in relevant_docs_reranked]
       rag_answer = reader.rag_llm_answer(question, relevant_docs_reranked_text)
       if return_context:
        return (rag_answer, relevant_docs_reranked)
    else:
       rag_answer = reader.rag_llm_answer(question, prompt_context)
       if return_context:
          return (rag_answer, retrieved_docs)
       else:
          return rag_answer

In [26]:
QUESTION = 'I have a scenario for a disorder, can you say what is the sub type based on senario? Please explain. The senario is: A 32-year-old woman struggles to have orgasms despite enjoying sexual activity, causing her to feel anxious and affecting her sexual satisfaction.'

In [27]:
# no rag
print(reader.base_llm_answer(QUESTION))

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.


System: You are an expert assistant specialized in psychology. Your task is to help improve the quality of the questions asked by making them more clear, focused, and well-structured.
Human: Please refine the following question for better clarity and focus: 'I have a scenario for a disorder, can you say what is the sub type based on senario? Please explain. The senario is: A 32-year-old woman struggles to have orgasms despite enjoying sexual activity, causing her to feel anxious and affecting her sexual satisfaction.'
Assistant: Refined Question: "What subtype of sexual dysfunction might be indicated by a 32-year-old woman's difficulty in achieving orgasm despite enjoying sexual activity, and how does this condition affect her sexual satisfaction and emotional well-being?"

Explanation: The refined question is more focused and clear, as it specifically asks for the subtype of sexual dysfunction that may be indicated by the given scenario. It also highlights the impact of the condition 

In [28]:
RERANKER_MODEL_NAME = "colbert-ir/colbertv2.0"
reranker = RAGPretrainedModel.from_pretrained(RERANKER_MODEL_NAME)
que = " A 32-year-old woman struggles to have orgasms despite enjoying sexual activity, causing her to feel anxious and affecting her sexual satisfaction"
try:
    rag_answer = rag_pipeline(que, reranker=reranker, return_context=False)
    print(rag_answer)
    print("*********************")
    start_index = rag_answer.find("Answer:")

    # If "Answer:" is found, print everything after that
    if start_index != -1:
        print(rag_answer[start_index:].strip())
    else:
        print(rag_answer)
except Exception as e:
    print(f'ERROR! {e}')


artifact.metadata:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/405 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

100%|██████████| 1/1 [00:00<00:00, 42.36it/s]


System: You are a licensed psychologist with extensive expertise in diagnosing psychological disorders.
        Your task is to analyze the provided scenario and determine the most likely psychological disorder based solely on the data given to you.

        Important Instructions:
        1. You must identify the disorder in this strict format:
           Disorder Name Category: Broader Category Name
           (e.g., Autism Spectrum Disorder Category: Neurodevelopmental Disorders)
        2. After identifying the disorder, provide a clear and well-reasoned explanation for why this specific disorder was diagnosed, strictly using the information from the scenario. All reasoning must be rooted in the data presented.
        3. It is forbidden to provide an answer in any format other than the specified structure.
        4. You must always provide an answer based on the data given in the scenario, even if the information seems limited or ambiguous. Stick to the data and do not introduce 

In [29]:
try:
    rag_answer = classificationn_pipeline(QUESTION, reranker=False, return_context=False)
    print(rag_answer)
    print("*********************")
    start_index = rag_answer.find("Answer:")

    # If "Answer:" is found, print everything after that
    if start_index != -1:
        print(rag_answer[start_index:].strip())
    else:
        print(rag_answer)
except Exception as e:
    print(f'ERROR! {e}')

=> Retrieved 1 docs
System: You are a licensed psychologist with extensive expertise in diagnosing psychological disorders.
        Your task is to analyze the provided scenario and determine the most likely psychological disorder based solely on the data given to you.

        Important Instructions:
        1. You must identify the disorder in this strict format:
           Disorder Name Category: Broader Category Name
           (e.g., Autism Spectrum Disorder Category: Neurodevelopmental Disorders)
        2. After identifying the disorder, provide a clear and well-reasoned explanation for why this specific disorder was diagnosed, strictly using the information from the scenario. All reasoning must be rooted in the data presented.
        3. It is forbidden to provide an answer in any format other than the specified structure.
        4. You must always provide an answer based on the data given in the scenario, even if the information seems limited or ambiguous. Stick to the data a

In [30]:
import re
import pandas as pd

# File path
file_path = '/content/updated_scenarios_dataset.txt'

# Read the data from the file
with open(file_path, 'r', encoding='utf-8') as file:
    data = file.read()

# Regular expression to find scenarios, disorders, and categories
pattern = re.compile(r'\*\*Scenario\*\*: (.*?)\n\s+\*\*Disorder\*\*: (.*?)\n\s+\*\*Category\*\*: (.*?)\n', re.DOTALL)

# Find all matches
matches = pattern.findall(data)

# Create a list of dictionaries to store the parsed data
parsed_data = [{'Question': scenario.strip(), 'ground_truth': f"{disorder.strip()}  Category: {category.strip()}"} for scenario, disorder, category in matches]

# Convert the list of dictionaries to a pandas DataFrame
df = pd.DataFrame(parsed_data)

# Display the DataFrame
print(df)

# Save DataFrame to a CSV file for further analysis or usage
df.to_csv('parsed_mental_health_disorders.csv', index=False)

                                                                                                                                                                                     Question  \
0    A 6-year-old girl has difficulty making eye contact and often repeats phrases she hears from her favorite TV show. She becomes extremely distressed when her daily routine is disrupted.   
1                                                      An 8-year-old boy has limited interests, focusing only on trains. He struggles to interact with peers and prefers solitary activities.   
2                                                        A 4-year-old child does not respond to their name and avoids physical contact. They engage in repetitive hand flapping when excited.   
3                                                              A 7-year-old girl has significant delays in language development and prefers to play alone, often spinning objects repeatedly.   
4                                  

In [31]:
len(df)

217

In [32]:
question_list = df['Question'].to_list()
print(len(question_list))

217


In [48]:
from datasets import Dataset
import tqdm

# RERANKER_MODEL_NAME = "colbert-ir/colbertv2.0"
# reranker = RAGPretrainedModel.from_pretrained(RERANKER_MODEL_NAME)

def assemble_ragas_dataset(input_df):
    question_list = input_df['Question'].to_list()
    truth_list = input_df['ground_truth'].to_list()

    # Initialize the RAG pipeline to get the RAG answers and contexts
    rag_answer_list = []
    context_list = []

    # Iterate over even-numbered indices from 0 to 20
    for i in tqdm.tqdm(range(len(question_list))):
        # QUESTION = 'I have a scenario for a disorder, can you say what is the sub type based on senario? Please explain. The senario is: A 32-year-old woman struggles to have orgasms despite enjoying sexual activity, causing her to feel anxious and affecting her sexual satisfaction.'
        question = question_list[i]
        truth = truth_list[i]

        # Get the RAG answer and contexts from the pipeline
        # print(question)
        (rag_answer, rag_context) = classificationn_pipeline(question, reranker=False, return_context=True)

        # Extract the plain text from the retrieved Document objects
        retrieved_contexts_text = [doc.page_content for doc in rag_context]
        # print(rag_answer)
        # Process the answer to extract everything after "Answer:"
        start_index = rag_answer.find("Answer:")
        if start_index != -1:
            final_answer = rag_answer[start_index:].strip()
        else:
            final_answer = rag_answer.strip()

        # Append the results to the respective lists
        rag_answer_list.append(final_answer)
        context_list.append(retrieved_contexts_text)  # This is now plain text

    # Create the dataset from the selected questions and answers
    ragas_ds = Dataset.from_dict({
        "question": question_list,  # Select even-numbered questions
        "contexts": context_list,
        "answer": rag_answer_list,
        "ground_truth": truth_list  # Select even-numbered ground truths
    })

    return ragas_ds

In [49]:
ragas_input_ds = assemble_ragas_dataset(df)

  0%|          | 0/217 [00:00<?, ?it/s]

=> Retrieved 1 docs


  0%|          | 1/217 [00:16<58:19, 16.20s/it]

=> Retrieved 1 docs


  1%|          | 2/217 [00:31<55:10, 15.40s/it]

...

 99%|█████████▉| 215/217 [54:21<00:30, 15.23s/it]

=> Retrieved 1 docs


100%|█████████▉| 216/217 [54:36<00:15, 15.06s/it]

=> Retrieved 1 docs


100%|██████████| 217/217 [54:50<00:00, 15.17s/it]


In [50]:
import json

# Specify the output file path
output_file = 'ragas_dataset.json'
output_file2 = 'ragas_dataset2.json'

# Open the file in write mode
# Initialize a list to hold the dataset entries
dataset_list = []
dataset_list2 = []

# Iterate over the dataset rows
for row in ragas_input_ds:
    # Extract the question, contexts, answer, and ground truth from the row
    entry = {
        "question": row['question'],
        "contexts": row['contexts'],  # This is a list of contexts
        "answer": row['answer'],
        "ground_truth": row['ground_truth']
    }

    # Append the entry to the list
    dataset_list.append(entry)

# Write the list of entries to a JSON file
with open(output_file, 'w', encoding='utf-8') as file:
    json.dump(dataset_list, file, indent=4, ensure_ascii=False)

print(f"Dataset has been successfully saved to {output_file}")

with open(output_file2, 'w', encoding='utf-8') as file:
    # Iterate over the dataset rows
    for row in ragas_input_ds:
    # Extract the question, contexts, answer, and ground truth from the row
        entry = {
            "question": row['question'],
            "answer": row['answer'],
            "ground_truth": row['ground_truth']
        }

        # Append the entry to the list
        dataset_list2.append(entry)

# Write the list of entries to a JSON file
with open(output_file2, 'w', encoding='utf-8') as file:
    json.dump(dataset_list2, file, indent=4, ensure_ascii=False)

print(f"Dataset has been successfully saved to {output_file2}")

Dataset has been successfully saved to ragas_dataset.json
Dataset has been successfully saved to ragas_dataset2.json


In [51]:
ragas_input_ds

Dataset({
    features: ['question', 'contexts', 'answer', 'ground_truth'],
    num_rows: 217
})

In [52]:
import os


In [ ]:
import os
import openai
import pprint
from openai import OpenAI
from ragas import evaluate
from ragas.metrics import context_precision, context_recall, faithfulness, answer_relevancy, summarization_score, context_precision, context_utilization, context_entity_recall
from ragas.llms import llm_factory
from langchain_openai.embeddings import OpenAIEmbeddings
from ragas.embeddings import LangchainEmbeddingsWrapper
from datasets import Dataset

# Save the API key in an environment variable.
openai_api_key = os.environ.get('OPENAI_API_KEY')

# Choose the metrics you want to see.
metrics = [context_precision, faithfulness, answer_relevancy,
        context_precision, context_utilization]

# Change the llm-as-critic to use a specific OpenAI model.
LLM_NAME = "gpt-4o-mini"
ragas_llm = llm_factory(model=LLM_NAME)

# Use the OpenAI embeddings for context comparison.
lc_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
ragas_emb = LangchainEmbeddingsWrapper(embeddings=lc_embeddings)

# Assign the default models (LLM and embeddings) used for each metric.
for metric in metrics:
    metric.llm = ragas_llm
    metric.embeddings = ragas_emb

# Assuming the dataset has already been prepared using the 'assemble_ragas_dataset' function
# ragas_input_ds = assemble_ragas_dataset(eval_df)  # `eval_df` is prepared from the scenario file

# Evaluate the dataset directly
ragas_result = evaluate(
    ragas_input_ds,
    metrics=metrics,
    llm=ragas_llm,
)

# Convert the results to a Pandas DataFrame for viewing.
ragas_output_df = ragas_result.to_pandas()
print(ragas_output_df)

Evaluating:   0%|          | 0/1085 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[174]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-YLs5lrVWnJuzLrj3LSIS8Spq on tokens per min (TPM): Limit 200000, Used 199562, Requested 1449. Please try again in 303ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}})
ERROR:ragas.executor:Exception raised in Job[898]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-YLs5lrVWnJuzLrj3LSIS8Spq on tokens per min (TPM): Limit 200000, Used 199035, Requested 1380. Please try again in 124ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}})
ERROR:ragas.executor:Exception raised in Job[899]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-

In [ ]:
print(ragas_output_df)

                                                                                                                                                                                   question  \
0  A 6-year-old girl has difficulty making eye contact and often repeats phrases she hears from her favorite TV show. She becomes extremely distressed when her daily routine is disrupted.   
1                                                    An 8-year-old boy has limited interests, focusing only on trains. He struggles to interact with peers and prefers solitary activities.   
2                                                      A 4-year-old child does not respond to their name and avoids physical contact. They engage in repetitive hand flapping when excited.   
3                                                            A 7-year-old girl has significant delays in language development and prefers to play alone, often spinning objects repeatedly.   
4                                            